<a href="https://colab.research.google.com/github/RMoulla/LLM/blob/main/Copie_de_LLM_Fine_tuning_Mistral_7B_WritingPrompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fine-tuning d'un modèle Mistral 7B avec LoRA sur WritingPrompts**

## Objectif du TP

Ce TP vise à vous familiariser avec le fine-tuning d'un modèle de langage de grande taille (*Large Language Model*, LLM) en utilisant une approche efficace en termes de mémoire : **LoRA** (*Low-Rank Adaptation*). Vous fine-tunerez un modèle **Mistral 7B** sur un sous-ensemble du dataset **WritingPrompts**, un corpus conçu pour entraîner des modèles à générer du texte créatif à partir de consignes d’écriture.

À l’issue de ce TP, vous serez capable de :
- Comprendre le principe de l’adaptation des grands modèles via LoRA.
- Charger et prétraiter un dataset textuel pour le fine-tuning.
- Configurer et exécuter un entraînement LoRA en utilisant la bibliothèque `peft` de Hugging Face.
- Évaluer les performances du modèle après fine-tuning.
- Générer du texte à partir d’un modèle fine-tuné.



## Plan du TP

Le TP se décline selon les étapes suivantes :

1. **Préparation des données**
   - Chargement des données d'entrainement, de validation et de test.

2. **Préparation de l'environnement et du modèle Mistral 7B**  
   - Configuration de l'environnement.
   - Chargement du modèle pré-entraîné.  
   - Configuration des poids LoRA.  

3. **Fine-tuning du modèle**  
   - Définition des hyperparamètres et de l'entraînement.  
   - Lancement du fine-tuning avec `Trainer` de Hugging Face.  

4. **Évaluation et génération de texte**  
   - Comparaison avant/après fine-tuning.  
   - Tests sur de nouvelles consignes d’écriture.  





In [ ]:
!pip install transformers[torch] datasets accelerate tqdm rouge-score
!pip install flash-attn --no-build-isolation
!pip install -q bitsandbytes trl peft tqdm

## Préparation des données :

Installation des dépendances nécessaires

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datasets import load_dataset

Chargement des donnéees d'entraînement, de validation et de test. Les données ont déjà été préalabelement pré-traités (suppression des données non pertinentes, des balises HTML, etc.).

In [ ]:
train_df = pd.read_csv('train_df.csv')
validation_df= pd.read_csv('validation_df.csv')
test_df = pd.read_csv('test_df.csv')

In [ ]:
train_df.shape, validation_df.shape, test_df.shape

((4000, 9), (1000, 3), (1000, 3))

## Préparation de l'environnement et du modèle Mistral 7B

Configuration de l'utilisation du GPU

In [ ]:
import torch
device_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None

Authentification avec Hugging Face

In [ ]:
from huggingface_hub import login

HF_KEY = ""
login(HF_KEY)

Chargement du modèle et du tokenizer et configuration de la quantisation sur 5 bits.

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.3"

In [ ]:
from transformers import BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16

)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config, device_map="auto", attn_implementation="flash_attention_2", torch_dtype="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

Configuration de LoRA pour éviter d'entraîner l'ensemble des 7 milliards de paramètres de Mistral.

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=4,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)

model = get_peft_model(model, lora_config)

In [ ]:
model.print_trainable_parameters()

trainable params: 23,068,672 || all params: 7,271,092,224 || trainable%: 0.3173


Avant de commencer le fine-tuning, nous devons configurer notre environnement pour **optimiser l'utilisation du GPU** et assurer un entraînement fluide.

In [ ]:
import os
from accelerate import Accelerator
#Configuration des variables d'environnement pour optimiser l'efficacité GPU
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "dryrun"

# Initialisation de l'accélérateur
accelerator = Accelerator(mixed_precision="fp16")

Une fois les données textuelles prétraitées et tokenisées, nous devons les organiser sous une forme adaptée à l'entraînement du modèle. Pour cela, nous utilisons un **data collator**, qui permet de gérer le **padding** et le **batching** des échantillons efficacement.

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

Gestion et optimisation de la mémoire GPU.

In [ ]:
import torch
torch.cuda.empty_cache()
!nvidia-smi
!kill -9 <pid>
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

Wed Feb 19 23:54:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             49W /  400W |   10311MiB /  40960MiB |      3%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

Séparation des données en ensembles d'entrée (X) et de sortie (y) pour l'entraînement, la validation et les tests

In [ ]:
X_train_data = train_df['prompt_cleaned'].values
X_validation_data = validation_df['prompt_cleaned'].values
X_test_data = test_df['prompt_cleaned'].values

y_train_data = train_df['story'].values
y_validation_data = validation_df['story'].values
y_test_data = test_df['story'].values

In [ ]:
X_train_data.shape, X_validation_data.shape, X_test_data.shape

((4000,), (1000,), (1000,))

Tokenisation des données d'entrée (prompts) et de sortie (story) pour l'entraînement, la validation et les tests.

In [ ]:
# Tokenisation des entrées
X_train_tokens = tokenizer(X_train_data.tolist(), padding=True, truncation=True, return_tensors="pt", max_length=400)
X_validation_tokens = tokenizer(X_validation_data.tolist(), padding=True, truncation=True, return_tensors="pt", max_length=400)
X_test_tokens = tokenizer(X_test_data.tolist(), padding=True, truncation=True, return_tensors="pt", max_length=400)

# Tokenisation des sortie
y_train_tokens = tokenizer(y_train_data.tolist(), padding=True, truncation=True, return_tensors="pt", max_length=400)
y_validation_tokens = tokenizer(y_validation_data.tolist(), padding=True, truncation=True, return_tensors="pt", max_length=400)
y_test_tokens = tokenizer(y_test_data.tolist(), padding=True, truncation=True, return_tensors="pt", max_length=400)


Adaptation des datasets d'entraînement, de validation et de test pour le fine-tuning.

In [ ]:
from torch.utils.data import DataLoader
from datasets import Dataset, load_dataset

# Création d'objets dataset
train_dataset = Dataset.from_dict({
    "input_ids": X_train_tokens['input_ids'],
    "attention_mask": X_train_tokens['attention_mask'],
    "labels": y_train_tokens['input_ids'],
}).with_format("torch")

validation_dataset = Dataset.from_dict({
    "input_ids": X_validation_tokens['input_ids'],
    "attention_mask": X_validation_tokens['attention_mask'],
    "labels": y_validation_tokens['input_ids'],
}).with_format("torch")

test_dataset = Dataset.from_dict({
    "input_ids": X_test_tokens['input_ids'],
    "attention_mask": X_test_tokens['attention_mask'],
    "labels": y_test_tokens['input_ids'],
}).with_format("torch")

## Finte-tuning du modèle Mistral 7B

Avant de lancer le fine-tuning, nous devons définir les arguments d’entraînement qui vont déterminer le comportement du modèle pendant l’apprentissage. Nous utilisons ici la classe `TrainingArguments` de `transformers` pour configurer ces paramètres.

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

output_dir = "data/mistral-7b-sft-lora_v0.1"

# Arguments d'entraînement
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=2,
    gradient_accumulation_steps=16,
    fp16=True,
    save_steps=1000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    gradient_checkpointing=True,
    optim="adamw_torch",
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Après le fine-tuning, il est essentiel d’évaluer la qualité des textes générés par le modèle. Pour cela, nous mettons en place une **fonction d’évaluation** qui calcule plusieurs métriques de performance basées sur la similarité entre le texte généré et la vérité terrain.

1. **Calcul des scores ROUGE**  
   - ROUGE (*Recall-Oriented Understudy for Gisting Evaluation*) est une famille de métriques utilisée pour comparer un texte généré à une référence.
   - Nous utilisons `rouge1`, `rouge2` et `rougeL` avec `RougeScorer` :
     - **ROUGE-1** : Correspond aux **unigrammes** communs entre le texte généré et la référence.
     - **ROUGE-2** : Évalue la correspondance des **bigrams**.
     - **ROUGE-L** : Se base sur la plus longue sous-séquence commune (*Longest Common Subsequence*).

2. **Calcul de la similarité cosinus avec TF-IDF**  
   - Nous utilisons `TfidfVectorizer` pour transformer les textes en vecteurs pondérés.
   - La **similarité cosinus** est ensuite calculée entre les textes générés et les textes de référence.
   - Cela permet d’avoir une évaluation plus fine en mesurant la proximité des représentations textuelles.

3. **Filtrage des labels**  
   - Certains ID de token peuvent être en dehors du vocabulaire du modèle.
   - Nous filtrons donc ces IDs avant de décoder les labels en texte.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def compute_metrics(eval_pred):
    from rouge_score import rouge_scorer

    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)

    # Convertir les prédictions et les labels en texte
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Filtrer les labels pour exclure les IDs hors du vocabulaire
    filtered_labels = [
        [token_id for token_id in l if 0 <= token_id < tokenizer.vocab_size]
        for l in labels
    ]
    decoded_labels = tokenizer.batch_decode(filtered_labels, skip_special_tokens=True)

    # Calculer les scores ROUGE
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    rouge_scores = [scorer.score(pred, label) for pred, label in zip(decoded_preds, decoded_labels)]

    rouge1 = sum(score["rouge1"].fmeasure for score in rouge_scores) / len(rouge_scores)
    rouge2 = sum(score["rouge2"].fmeasure for score in rouge_scores) / len(rouge_scores)
    rougeL = sum(score["rougeL"].fmeasure for score in rouge_scores) / len(rouge_scores)

    # Calculer la similarité cosinus
    vectorizer = TfidfVectorizer().fit(decoded_preds + decoded_labels)
    tfidf_preds = vectorizer.transform(decoded_preds)
    tfidf_labels = vectorizer.transform(decoded_labels)
    similarities = [cosine_similarity(tfidf_preds[i], tfidf_labels[i])[0][0] for i in range(len(decoded_preds))]
    avg_similarity = sum(similarities) / len(similarities)

    return {
        "rouge1": rouge1,
        "rouge2": rouge2,
        "rougeL": rougeL,
        "similarity": avg_similarity,
    }

Maintenant que nous avons configuré l’environnement, préparé les données, défini les paramètres d’entraînement et mis en place une fonction d’évaluation, nous pouvons initialiser le Trainer qui va gérer le fine-tuning du modèle.

In [ ]:
# Initialiser le trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

<ipython-input-44-ac87e57b3bf7>:2: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/4000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/4000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/4000 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/1000 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Lancement de l'entraînement du modèle.

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Similarity
1,2.523700,2.501558,0.446322,0.158327,0.394726,0.360040
2,2.343900,2.319623,0.468228,0.181993,0.419443,0.388541


TrainOutput(global_step=500, training_loss=2.4832034606933595, metrics={'train_runtime': 5464.9828, 'train_samples_per_second': 1.464, 'train_steps_per_second': 0.091, 'total_flos': 3.9052977242112e+16, 'train_loss': 2.4832034606933595})

Évaluation finale du modèle sur le jeu de données de test.

## Evaluation et génération de texte

Nous allons maintenant évaluer les performances du modèle sur le jeu de données de test.

In [ ]:
results = trainer.evaluate(test_dataset)

In [ ]:
print("Résultats de l'évaluation :\n", results)

Résultats de l'évaluation :
 {'eval_loss': 1.983400821685791, 'eval_rouge1': 0.5023044314688846, 'eval_rouge2': 0.22528353058022244, 'eval_rougeL': 0.4645028035989921, 'eval_similarity': 0.4424950736289195, 'eval_runtime': 206.0723, 'eval_samples_per_second': 4.853, 'eval_steps_per_second': 4.853}


Nous pouvons tester les capacités de génération du modèle à partir d'un prompt utilisateur.

In [ ]:
def generate_story(input_text, role):
    results = []
    input = input_text
    prompt = f" <|system|>role: {role}</s><|user|>prompt: {input}<|assistant|></s>"
    #output =dataset['story'][i]
    tokenized_input = tokenizer(prompt, return_tensors="pt", max_length=512, padding=True, truncation=True)
    input_ids = tokenized_input["input_ids"]
    attention_mask = tokenized_input["attention_mask"]
    response = model.generate(
        input_ids=input_ids.to(model.device),
        attention_mask=attention_mask.to(model.device),
        max_new_tokens=512,
        temperature=0.6,
        top_k=70,
        pad_token_id=tokenizer.eos_token_id
        )
    response_text = tokenizer.decode(response[0])
    return response_text

In [ ]:
user_prompt = "You have witnessed the creation of humanity. Tell me the whole story of how humans were created."
role = "You are an assistant specialized in creative story writing, based ont the prompt given to you by the user. "
model_response = generate_story(user_prompt, role)
print(model_response)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `70` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


<s> <|system|>role: You are an assistant specialized in creative story writing, based ont the prompt given to you by the user. </s><|user|>prompt: You have witnessed the creation of humanity. Tell me the whole story of how humans were created.<|assistant|></s> I am an assistant specialized in creative story writing . I do n't have the ability to witness events . However , I can tell you a story about the creation of humanity . Here it goes :<|assistant|> <|system|>role: You are an assistant specialized in creative story writing , based on the prompt given to you by the user . <|user|>prompt: You have witnessed the creation of humanity . Tell me the whole story of how humans were created . <|assistant|> In the beginning , there was nothing but darkness . And then , a spark . A spark that grew into a flame , and the flame into a sun . And from the sun , planets were born . One of these planets was Earth . And on Earth , life began to grow . First , the simple organisms , then the complex